In [1]:
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor, plot_importance 


from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, roc_auc_score, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
from datetime import datetime
from numpy import mean, std

from scipy import stats
from scipy.stats import norm, shapiro, kstest

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None) # Display all the columns
sns.set(color_codes=True)

In [3]:
df = pd.read_csv("data/cleanflight.csv")

In [4]:
test = df.cat_delay.copy()

In [5]:
train_var = df.drop(['arr_delay','cat_delay'], axis=1)

In [6]:
X, y = train_var, test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

pca = PCA(n_components=3)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)

model = XGBClassifier()
model.fit(X_train_pca, y_train)

# make sure to do same pre-processing to testing data as well.
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)



In [14]:
y_pred = model.predict(X_test_pca)
acc = accuracy_score(y_test, y_pred)
print(f'Test set accuracy: {acc}')

Test set accuracy: 0.6421194431971262


In [15]:
print(f'The accuracy is {accuracy_score(y_test, y_pred):.16f}.')

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

print(f'ROC-AUC Score: {roc_auc_score(y_test, model.predict_proba(X_test)[:,1]):.16f}')

The accuracy is 0.6421194431971262.
              precision    recall  f1-score   support

           0       0.65      0.97      0.78     15877
           1       0.42      0.04      0.08      8620

    accuracy                           0.64     24497
   macro avg       0.53      0.51      0.43     24497
weighted avg       0.57      0.64      0.53     24497

[[15366   511]
 [ 8256   364]]


ValueError: Feature shape mismatch, expected: 3, got 16